In [3]:
from components.common import load_file_lists
%pwd

'/mnt/c/Users/kubar/Documents/Sem7/labelthissongforme'

In [2]:
%cd ../

/mnt/c/Users/kubar/Documents/Sem7/labelthissongforme


In [4]:
import os
import random
import numpy as np
import torch

from sklearn.model_selection import train_test_split

In [8]:
split_path = "split/mtat-20/"
os.makedirs(split_path, exist_ok=True)

In [9]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(123456)

Random seed set as 123456


In [10]:
# p = OpenL3PreProcessor(input_path="../data/mtat/mp3",
#                        output_path="../data/mtat/emb",
#                        suffix="npy")
# # print(load_file_lists(["../split/mtat/train.npy", "../split/mtat/valid.npy", "../split/mtat/test.npy"])[:, 1])
data = load_file_lists([
    os.path.join(split_path, "train.npy"),
    os.path.join(split_path, "valid.npy"),
    os.path.join(split_path, "test.npy")
])
# p.run(files=data[:, 1])

In [11]:
binary = {row[0]: row[1:] for row in np.load(os.path.join(split_path, "binary.npy"), allow_pickle=True)}
tags = np.load(os.path.join(split_path, "tags.npy"), allow_pickle=True)
tags

array(['guitar', 'classical', 'slow', 'techno', 'strings', 'drums',
       'electronic', 'rock', 'fast', 'piano', 'ambient', 'beat', 'violin',
       'vocal', 'synth', 'female', 'indian', 'opera', 'male', 'singing'],
      dtype='<U10')

In [15]:
import pathlib

X = []
Y = []
for idx, filename in data:
    filename = os.path.join("data/mtat/emb", str(pathlib.Path(filename).with_suffix(".npy")))
    file_data = np.load(filename, allow_pickle=True).flatten()
    X.append(file_data)
    Y.append(binary[int(idx)])
X_np = np.array(X)
np.save("X.npy", X_np)
Y_np = np.array(Y)
np.save("y.npy", Y_np)

In [16]:
X_np = np.load("X.npy", allow_pickle=True)
X_np

array([[2.6140246 , 1.8475785 , 4.3649316 , ..., 1.1230563 , 3.333457  ,
        2.867814  ],
       [2.4309678 , 2.4298744 , 2.9125767 , ..., 1.5540198 , 3.5779722 ,
        3.57896   ],
       [2.5092711 , 2.2790258 , 2.4283175 , ..., 0.48853248, 3.4483852 ,
        2.2871022 ],
       ...,
       [2.4309678 , 2.2870908 , 3.9133983 , ..., 2.0478022 , 3.410993  ,
        3.0003796 ],
       [2.4309678 , 2.036916  , 3.4007561 , ..., 1.6041538 , 3.2292883 ,
        3.3360808 ],
       [2.4309678 , 2.7142615 , 2.9608872 , ..., 1.4507025 , 4.896281  ,
        3.2105606 ]], dtype=float32)

In [17]:
Y_np = np.load("Y.npy", allow_pickle=True)
Y_np

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X_np, Y_np, random_state=1)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(bootstrap=True,
                             max_depth=20,
                             max_features='sqrt',
                             n_jobs=4,
                             random_state=1,
                             warm_start=True)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.47      0.92      0.62       612
           1       0.59      0.78      0.67       785
           2       0.07      0.61      0.13       101
           3       0.54      0.83      0.65       487
           4       0.14      0.60      0.22       149
           5       0.11      0.80      0.19        88
           6       0.07      0.57      0.13        81
           7       0.57      0.98      0.72       353
           8       0.08      0.66      0.14        68
           9       0.35      1.00      0.51       184
          10       0.32      0.90      0.47       176
          11       0.15      0.71      0.24       105
          12       0.36      0.78      0.49       195
          13       0.01      0.75      0.01         4
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.01      1.00      0.02         4
          17       0.39    

/home/robak/.virtualenvs/labelthissongforme/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/robak/.virtualenvs/labelthissongforme/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

model = DecisionTreeClassifier(max_depth=20,
                               max_features='sqrt',
                               random_state=1)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.53      0.51      0.52      1222
           1       0.55      0.53      0.54      1080
           2       0.28      0.28      0.28       894
           3       0.54      0.55      0.55       730
           4       0.37      0.36      0.37       687
           5       0.29      0.30      0.29       636
           6       0.32      0.31      0.32       648
           7       0.56      0.57      0.56       596
           8       0.26      0.26      0.26       566
           9       0.39      0.43      0.41       478
          10       0.39      0.40      0.39       484
          11       0.29      0.35      0.32       433
          12       0.43      0.40      0.41       448
          13       0.23      0.21      0.22       495
          14       0.18      0.18      0.18       411
          15       0.20      0.19      0.19       384
          16       0.20      0.20      0.20       351
          17       0.52    

/home/robak/.virtualenvs/labelthissongforme/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

model = KNeighborsClassifier()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.66      0.79      0.72       987
           1       0.73      0.70      0.71      1091
           2       0.26      0.50      0.34       456
           3       0.72      0.72      0.72       749
           4       0.40      0.53      0.46       494
           5       0.41      0.54      0.47       506
           6       0.31      0.51      0.39       379
           7       0.74      0.88      0.81       510
           8       0.40      0.43      0.41       540
           9       0.56      0.79      0.66       383
          10       0.63      0.68      0.65       464
          11       0.40      0.47      0.43       435
          12       0.62      0.70      0.66       378
          13       0.12      0.47      0.19       115
          14       0.17      0.42      0.25       178
          15       0.28      0.65      0.39       161
          16       0.52      0.65      0.58       274
          17       0.81    

/home/robak/.virtualenvs/labelthissongforme/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.metrics import classification_report

model = RadiusNeighborsClassifier(radius=100.)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.02      0.96      0.04        24
           1       0.00      0.11      0.00         9
           2       0.00      0.00      0.00         0
           3       0.09      0.85      0.16        75
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           6       0.00      0.50      0.00         2
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.01      1.00      0.01         4
          10       0.00      1.00      0.01         2
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         0
          17       0.00    

/home/robak/.virtualenvs/labelthissongforme/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/robak/.virtualenvs/labelthissongforme/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import classification_report

single_output_model = LogisticRegression(solver='saga',
                                         max_iter=3000,
                                         random_state=1)
model = ClassifierChain(single_output_model, random_state=1)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

single_output_model = LogisticRegression(solver='saga',
                                         max_iter=3000,
                                         random_state=1)
model = MultiOutputClassifier(single_output_model, n_jobs=4)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))

In [ ]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

single_output_model = SVC(max_iter=3000, random_state=1)
model = MultiOutputClassifier(single_output_model, n_jobs=4)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))